# Using jupyter-jchannel in a local notebook

## Installing

You can use your favorite manager to install the [PyPI package](https://pypi.org/project/jupyter-jchannel/).

```
pip install jupyter-jchannel
```

## Starting a server

For basic usage, you only need to import the `jchannel` module.

In [1]:
import jchannel

A [jchannel Server](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server) instance runs alongside a Jupyter server instead of over it. Therefore, it needs its own local address.

In [2]:
HOST = '127.0.0.1'
PORT = 8889

The asynchronous [start function](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.html#jchannel.start) instantiates a server, starts this server and returns it.

In [ ]:
server = await jchannel.start(host=HOST, port=PORT)

## Opening a channel

A [server channel](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel) uses a server to call frontend JavaScript code from kernel Python code. The asynchronous [open method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.server.html#jchannel.server.Server.open) instantiates a channel, opens this channel and returns it.

This method receives a string representing a JavaScript function. This function should receive a [client representation of the same channel](https://hashiprobr.github.io/jupyter-jchannel-client/Channel.html) and initialize it. The most important part of this initialization is setting the `handler` property to an object. The methods of this object define the API at the frontend.

The example below sets `handler` to an object that wraps the [padStart](https://developer.mozilla.org/docs/Web/JavaScript/Reference/Global_Objects/String/padStart) and [padEnd](https://developer.mozilla.org/docs/Web/JavaScript/Reference/Global_Objects/String/padEnd) string methods to provide indentation utilities.

In [4]:
channel1 = await server.open('''
    (channel) => {
        channel.handler = {
            indentLeft(line, count) {
                return line.padStart(line.length + count);
            },
            indentRight(line, count) {
                return line.padEnd(line.length + count);
            },
        };
    }
''')

And that's it! You can now call any method of this API from the notebook.

## Calling the methods

The [call method](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel.call) returns a [task](https://docs.python.org/3/library/asyncio-task.html#asyncio.Task) that can be awaited to retrieve the result whenever you want.

In [5]:
taskA = channel1.call('indentLeft', 'hello', 4)
taskB = channel1.call('indentRight', 'world', 4)

In [6]:
# some code you want to run before retrieving the results

In [7]:
await taskB

'world    '

In [8]:
await taskA

'    hello'

In particular, awaiting immediately ensures synchronous execution, without the
need for sleeping and/or polling.

In [9]:
await channel1.call('indentLeft', 'hello', 4)

'    hello'

Furthermore, if the frontend throws a JavaScript exception, the task wraps it into a Python exception.

In [10]:
await channel1.call('indentRight', 4, 'world') # arguments in wrong order

FrontendError: TypeError: line.padEnd is not a function

## Example 2: calling kernel from frontend

If you want to call kernel Python code from frontend JavaScript code, the first part is storing the client representation of the channel in order to retrieve it later.

The example below simply stores it as a global variable.

In [11]:
channel2 = await server.open('''
    (channel) => {
        self.channel2 = channel;
    }
''')

The second part is setting the `handler` property to an object. The methods of this object define the API at the kernel.

In [12]:
class Handler:
    def multiply(self, a, b):
        return a * b

    def divide(self, a, b):
        return a / b

In [13]:
channel2.handler = Handler()

And that's it! You can now call any method of this API from the console.

![](https://raw.githubusercontent.com/hashiprobr/jupyter-jchannel/main/docs/images/console_capture_2.png)

## Example 3: binary streams

Now, suppose you want to send a binary stream as an argument. For example, the bytes of a text file.

In [14]:
with open('loremipsum.txt') as file:
    print(file.read())

Lorem ipsum dolor sit amet,
consectetur adipiscing elit,
sed do eiusmod tempor incididunt
ut labore et dolore magna aliqua.



The example below defines an async method that receives an async iterable of `Uint8Arrays` and returns the total of bytes.

In [15]:
channel3 = await server.open('''
    (channel) => {
        channel.handler = {

            async len(stream) {
                let total = 0;

                for await (const chunk of stream) {
                    total += chunk.length;
                }

                return total;
            },

        };
    }
''')

You cannot use `call` in this case, but you can use [call_with_stream](https://jupyter-jchannel.readthedocs.io/en/latest/jchannel.channel.html#jchannel.channel.Channel.call_with_stream). It's a version that sends an async iterable of bytes-like objects as the first argument. For example, an async generator of encoded lines.

In [16]:
async def input_stream():
    with open('loremipsum.txt') as file:
        for line in file:
            yield line.encode()

In [17]:
await channel3.call_with_stream('len', input_stream())

124

What's more, you can receive a binary stream as the return value!

The example below defines an async generator method that decodes the received bytes, converts the corresponding characters to uppercase, encodes the converted characters and yields them.

In [18]:
channel4 = await server.open('''
    (channel) => {
        channel.handler = {

            async * upper(stream) {
                const decoder = new TextDecoder();
                const encoder = new TextEncoder();

                for await (const chunk of stream) {
                    const line = decoder.decode(chunk);
                    yield encoder.encode(line.toUpperCase());
                }
            },

        };
    }
''')

In this case, both `call` and `call_with_stream` receive an async iterable of bytes objects as the return value.

In [19]:
output_stream = await channel4.call_with_stream('upper', input_stream())

In [20]:
async for chunk in output_stream:
    print(chunk.decode())

LOREM IPSUM DOLOR SIT AMET,
CONSECTETUR ADIPISCING ELIT,
SED DO EIUSMOD TEMPOR INCIDIDUNT
UT LABORE ET DOLORE MAGNA ALIQUA.

